In [25]:
import os
import cv2
import time
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor

def detect_and_match_features(image_paths):
    # Initialize OpenCV feature detector (SIFT, ORB, etc.) and matcher (Brute Force, FLANN, etc.)
    detector = cv2.SIFT_create()
    matcher = cv2.BFMatcher()

    # Function to detect keypoints and compute descriptors for a single image
    def detect_and_compute(image_path):
        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Error reading image {image_path}. Skipping.")
                return None, None
            keypoints, descriptors = detector.detectAndCompute(image, None)
            if keypoints is None or descriptors is None:
                print(f"Error detecting features in image {image_path}. Skipping.")
                return None, None
            return keypoints, descriptors
        except Exception as e:
            print(f"Exception during feature detection: {e}. Skipping {image_path}.")
            return None, None

    # Detect keypoints and compute descriptors for all images in parallel
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(detect_and_compute, image_paths))

    # Separate keypoints and descriptors from results
    keypoints_list, descriptors_list = zip(*results)
    keypoints_list = [keypoints for keypoints in keypoints_list if keypoints is not None]
    descriptors_list = [descriptors for descriptors in descriptors_list if descriptors is not None]

    # Match features between images
    matches = [matcher.match(descriptors_list[i], descriptors_list[i + 1]) for i in range(len(descriptors_list) - 1)]

    return keypoints_list, matches

def find_image_paths(root_dir, scene_category):
    image_paths = []
    scene_dir = os.path.join(root_dir, scene_category.replace(" ", "_"))
    if os.path.isdir(scene_dir):
        for subdir, _, files in os.walk(scene_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(subdir, file))
    return image_paths

def visualize_matches(image_paths, keypoints_list, matches):
    for i, match in enumerate(matches):
        try:
            # Read images for the current pair of matches
            image1 = cv2.imread(image_paths[i], cv2.IMREAD_GRAYSCALE)
            image2 = cv2.imread(image_paths[i + 1], cv2.IMREAD_GRAYSCALE)

            # Get keypoints for the current pair of images
            keypoints1 = keypoints_list[i]
            keypoints2 = keypoints_list[i + 1]

            # Draw matches between keypoints on the images
            matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, match, None)
            
            # Display the matched image
            plt.figure(figsize=(15, 7))
            plt.title(f"Matched Images {i + 1} to {i + 2}")
            plt.imshow(matched_image)
            plt.show()
        except Exception as e:
            print(f"Exception during match visualization: {e}. Skipping visualization for image pair {i}.")

if __name__ == "__main__":
    # Specify the root directory containing images
    root_dir = "C:/Vishal/Kaggele-Image-Matching-Challenge-2024/data/train"

    # Define the scene categories
    scene_categories = [
        "church",
        "dioscuri",
        "lizard",
        "multi-temporal-temple-baalshamin",
        "pond",
        "transp_obj_glass_cup",
        "transp_obj_glass_cylinder"
    ]

    # Iterate over each scene category
    for scene_category in scene_categories:
        print(f"Processing images for scene category: {scene_category}")

        # Start timing
        start_time = time.time()

        # Find image paths for the current scene category
        image_paths = find_image_paths(root_dir, scene_category)

        if not image_paths:
            print("No images found for this scene category.")
        else:
            # Detect and match features using OpenCV
            keypoints_list, matches = detect_and_match_features(image_paths)

            # Visualize matches
            #visualize_matches(image_paths, keypoints_list, matches)

        # End timing
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Time taken for scene category '{scene_category}': {elapsed_time:.2f} seconds")
        print("------------------------------------------------------------------")


In [33]:
import os
import cv2
import time
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor


def detect_and_match_features(image_paths, scale_factor=0.5, detector_name="SIFT"):
    """
    Detects and matches features for a list of images with optimization options.

    Args:
        image_paths (list): List of image paths.
        scale_factor (float, optional): Factor for image downscaling (default: 0.5).
        detector_name (str, optional): Name of the feature detector (default: "SIFT").

    Returns:
        tuple: A tuple containing lists of keypoints and matches.
    """

    # Choose detector based on name (can be extended for other detectors)
    if detector_name == "SIFT":
        detector = cv2.SIFT_create()
    elif detector_name == "ORB":
        detector = cv2.ORB_create()
    else:
        raise ValueError(f"Unsupported detector name: {detector_name}")

    matcher = cv2.BFMatcher()

    def detect_and_compute(image_path):
        try:
            # Read and downscale image (if applicable)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Error reading image {image_path}. Skipping.")
                return None, None
            image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

            # Detect keypoints and compute descriptors
            keypoints, descriptors = detector.detectAndCompute(image, None)
            if keypoints is None or descriptors is None:
                print(f"Error detecting features in image {image_path}. Skipping.")
                return None, None
            return keypoints, descriptors
        except Exception as e:
            print(f"Exception during feature detection: {e}. Skipping {image_path}.")
            return None, None

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(detect_and_compute, image_paths))

    # Separate keypoints and descriptors from results
    # (filter out None values from skipped images)
    keypoints_list = [keypoints for keypoints, _ in results if keypoints is not None]
    descriptors_list = [descriptors for _, descriptors in results if descriptors is not None]

    # Match features between images using brute-force matcher
    matches = []
    for i in range(len(descriptors_list) - 1):
        matches.append(matcher.match(descriptors_list[i], descriptors_list[i + 1]))

    return keypoints_list, matches


In [34]:
# Main fuction
if __name__ == "__main__":
    # Specify the root directory containing images
    root_dir = "C:/Vishal/Kaggele-Image-Matching-Challenge-2024/data/train"

    # Define the scene categories
    scene_categories = [
        "church",
        "dioscuri",
        "lizard",
        "multi-temporal-temple-baalshamin",
        "pond",
        "transp_obj_glass_cup",
        "transp_obj_glass_cylinder"
    ]

    # Iterate over each scene category
    for scene_category in scene_categories:
        print(f"Processing images for scene category: {scene_category}")

        # Start timing
        start_time = time.time()

        # Find image paths for the current scene category
        image_paths = find_image_paths(root_dir, scene_category)

        if not image_paths:
            print("No images found for this scene category.")
        else:
            # Detect and match features using OpenCV with brute-force matcher
            keypoints_list, matches = detect_and_match_features(image_paths)

            # Visualize matches
            # visualize_matches(image_paths, keypoints_list, matches)

        # End timing
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Time taken for scene category '{scene_category}': {elapsed_time:.2f} seconds")
        print("------------------------------------------------------------------")


Processing images for scene category: church
Time taken for scene category 'church': 7.91 seconds
------------------------------------------------------------------
Processing images for scene category: dioscuri
Time taken for scene category 'dioscuri': 5.42 seconds
------------------------------------------------------------------
Processing images for scene category: lizard
Time taken for scene category 'lizard': 42.75 seconds
------------------------------------------------------------------
Processing images for scene category: multi-temporal-temple-baalshamin
Time taken for scene category 'multi-temporal-temple-baalshamin': 20.62 seconds
------------------------------------------------------------------
Processing images for scene category: pond
Time taken for scene category 'pond': 117.22 seconds
------------------------------------------------------------------
Processing images for scene category: transp_obj_glass_cup
Time taken for scene category 'transp_obj_glass_cup': 88.67 

KeyboardInterrupt: 

In [1]:
def parse_camera_data(camera_file):
    """Parse camera data from a file.
    
    Args:
        camera_file (str): Path to the camera data file.
        
    Returns:
        dict: A dictionary containing camera data with camera ID as keys.
              Each camera entry contains model, width, and height.
    """
    camera_data = {}
    try:
        with open(camera_file, 'r') as file:
            lines = file.readlines()
            for line in lines[3:]:  # Skip the first three comment lines
                parts = line.strip().split(' ')
                camera_id = int(parts[0])
                model = parts[1]
                width = int(parts[2])
                height = int(parts[3])
                camera_data[camera_id] = {'model': model, 'width': width, 'height': height}
    except FileNotFoundError:
        print(f"Error: File '{camera_file}' not found.")
    except Exception as e:
        print(f"Error while parsing camera data: {e}")
    return camera_data

def parse_image_data(image_file):
    """Parse image data from a file.
    
    Args:
        image_file (str): Path to the image data file.
        
    Returns:
        dict: A dictionary containing image data with image ID as keys.
              Each image entry contains camera ID and name.
    """
    image_data = {}
    try:
        with open(image_file, 'r') as file:
            lines = file.readlines()
            for i in range(0, len(lines), 2):  # Process every two lines
                image_info = lines[i].strip().split(' ')
                image_id = int(image_info[0])
                camera_id = int(image_info[-2])
                name = image_info[-1]
                image_data[image_id] = {'camera_id': camera_id, 'name': name}
    except FileNotFoundError:
        print(f"Error: File '{image_file}' not found.")
    except Exception as e:
        print(f"Error while parsing image data: {e}")
    return image_data

def parse_point3d_data(points3d_file):
    """Parse 3D point data from a file.
    
    Args:
        points3d_file (str): Path to the 3D point data file.
        
    Returns:
        dict: A dictionary containing 3D point data with point ID as keys.
              Each point entry contains x, y, and z coordinates.
    """
    point3d_data = {}
    try:
        with open(points3d_file, 'r') as file:
            lines = file.readlines()
            for line in lines[3:]:  # Skip the first three comment lines
                parts = line.strip().split(' ')
                point3d_id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                z = float(parts[3])
                point3d_data[point3d_id] = {'x': x, 'y': y, 'z': z}
    except FileNotFoundError:
        print(f"Error: File '{points3d_file}' not found.")
    except Exception as e:
        print(f"Error while parsing 3D point data: {e}")
    return point3d_data

In [5]:
def geometric_verification(matches, keypoints1, keypoints2, camera_poses):
    """Performs geometric verification of feature matches using epipolar constraint.

    Args:
        matches (list): List of matches between keypoints.
        keyframes1 (list): List of keypoints for the first image.
        keyframes2 (list): List of keypoints for the second image.
        camera_poses (dict): Dictionary containing camera poses for each image.

    Returns:
        list: A list of verified matches that pass the epipolar constraint.
    """

    verified_matches = []
    fx = camera_poses[image_paths[0]]['fx']  # Assuming focal length is available in camera data
    fy = camera_poses[image_paths[0]]['fy']  # Assuming focal length is available in camera data
    cx = camera_poses[image_paths[0]]['cx']  # Assuming principal point is available in camera data
    cy = camera_poses[image_paths[0]]['cy']  # Assuming principal point is available in camera data

    for match in matches:
        # Extract keypoint indices from the match
        query_idx = match.queryIdx
        train_idx = match.trainIdx

        # Get keypoint locations from keypoint lists
        query_pt = keypoints1[query_idx].pt
        train_pt = keypoints2[train_idx].pt

        # Extract camera poses for the image pair
        camera_pose1 = camera_poses[image_paths[0]]
        camera_pose2 = camera_poses[image_paths[1]]

        # Convert camera orientations from quaternions to rotation matrices
        R1 = R.from_quat(camera_pose1['orientation'])
        R2 = R.from_quat(camera_pose2['orientation'])

        # Calculate 3D world point corresponding to the query keypoint in the first image
        world_pt = np.linalg.inv(np.dot(R1.as_matrix(), np.array([[camera_poses[image_paths[0]]['location'][0],
                                                                   camera_poses[image_paths[0]]['location'][1],
                                                                   camera_poses[image_paths[0]]['location'][2],
                                                                   1]]).T)) * np.dot(R1.as_matrix(), np.array([[(query_pt[0] - cx) / fx],
                                                                                                   [(query_pt[1] - cy) / fy],
                                                                                                   [1],
                                                                                                   [0]]).T)

        # Project the 3D world point back onto the second image plane using the second camera pose
        projected_pt = np.dot(R2.as_matrix().T, (np.dot(camera_pose2['calibration_matrix'], world_pt) - np.array([camera_pose2['location'][0], camera_pose2['location'][1], camera_pose2['location'][2], 1]).T))[:2] / projected_pt[2]

        # Check if the distance between the projected point and the train point is within

In [7]:
def refine_matches_lowe(matches, descriptors1, descriptors2):
    """Refines feature matches using Lowe's ratio test.

    Args:
        matches (list): List of matches between keypoints.
        descriptors1 (list): List of descriptors for the first image.
        descriptors2 (list): List of descriptors for the second image.

    Returns:
        list: A list of refined matches that pass the Lowe's ratio test.
    """

    refined_matches = []
    ratio_thresh = 0.8  # Adjust threshold as needed

    for match in matches:
        # Get the two best matches for the query descriptor
        first_match = match
        second_match_idx = min(i for i in range(len(matches)) if i != match.queryIdx and matches[i].trainIdx == match.trainIdx)
        second_match = matches[second_match_idx]

        # Calculate the distance ratio
        distance_ratio = match.distance / second_match.distance

        # If the ratio is greater than the threshold, consider it a good match
        if distance_ratio > ratio_thresh:
            refined_matches.append(match)

    return refined_matches

In [8]:
def filter_matches_ransac(matches, keypoints1, keypoints2, camera_poses, ransac_thresh=3.0):
  """
  Filters feature matches using RANSAC (Random Sample Consensus).

  Args:
      matches (list): List of matches between keypoints.
      keyframes1 (list): List of keypoints for the first image.
      keyframes2 (list): List of keypoints for the second image.
      camera_poses (dict): Dictionary containing camera poses for each image.
      ransac_thresh (float, optional): Threshold for inlier distance in RANSAC. Defaults to 3.0.

  Returns:
      list: A list of inlier matches that pass the RANSAC test.
  """

  # Import necessary libraries (assuming OpenCV is already imported)
  import cv2

  inlier_matches = []
  if len(matches) > 4:  # Require at least 4 matches for RANSAC
      # Extract keypoint locations for all matches
      src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
      dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

      # Estimate homography (or fundamental matrix for uncalibrated cameras)
      model, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransac_thresh)

      # Filter inliers based on the homography and distance threshold
      for i, m in enumerate(matches):
          if mask[i][0]:  # Check if the match is an inlier
              # Project the source point back onto the second image using the homography
              projected_pt = cv2.perspectiveTransform(src_pts[i].reshape(-1, 1, 2), model)
              # Calculate the distance between the projected point and the matched point
              distance = np.linalg.norm(projected_pt[0][0] - dst_pts[i])
              # Add the inlier match if the distance is within a threshold
              if distance < ransac_thresh:
                  inlier_matches.append(m)

  return inlier_matches

In [9]:
def triangulate_points(inlier_matches, keypoints1, keypoints2, camera_poses):
  """
  Reconstructs 3D points from inlier matches using triangulation.

  Args:
      inlier_matches (list): List of inlier matches after RANSAC filtering.
      keyframes1 (list): List of keypoints for the first image.
      keyframes2 (list): List of keypoints for the second image.
      camera_poses (dict): Dictionary containing camera poses for each image.

  Returns:
      list: A list of reconstructed 3D points.
  """

  # Import libraries (assuming NumPy is already imported)
  import numpy as np

  # Initialize empty list for 3D points
  points3d = []

  for match in inlier_matches:
      # Extract corresponding keypoints from both images
      query_pt = keypoints1[match.queryIdx].pt
      train_pt = keypoints2[match.trainIdx].pt

      # Extract camera projection matrices from camera poses
      P1 = camera_poses[match.imgIdx1]  # Assuming camera poses are stored by image index
      P2 = camera_poses[match.imgIdx2]

      # Homogeneous coordinates for the corresponding keypoints
      q = np.array([query_pt[0], query_pt[1], 1.0])
      p = np.array([train_pt[0], train_pt[1], 1.0])

      # Triangulation using linear least squares solution
      A = np.vstack([np.cross(P1[2], P1[1] * p[0] - P1[0] * p[1]),
                     np.cross(P2[2], P2[1] * q[0] - P2[0] * q[1])])
      _, _, V = np.linalg.svd(A)
      X = V[-1, :] / V[-1, -1]  # Normalized homogeneous coordinates of the 3D point

      # Convert to non-homogeneous coordinates
      points3d.append(X[:3])

  return points3d

In [12]:
def poisson_surface_mesh(points3d):
  """
  Generates a surface mesh from a set of 3D points using Poisson surface reconstruction.

  Args:
      points3d (list): List of reconstructed 3D points.

  Returns:
      tuple: A tuple containing the vertices and faces of the surface mesh.
  """

  # Import libraries (assuming PyMesh is already installed)
  try:
    import pymesh
  except ImportError:
    print("PyMesh library not found. Please install it for surface meshing functionality.")
    return None

  # Convert points to PyMesh format (if PyMesh is available)
  if pymesh:
    verts = pymesh.utils.trimesh_to_vertex_list(np.array(points3d))

    # Perform Poisson surface reconstruction
    mesh = pymesh.generate_poisson_mesh(verts)

    # Extract vertices and faces
    vertices = mesh.vertices
    faces = mesh.faces

    return vertices, faces
  else:
    return None

In [13]:
def visualize_reconstruction(points3d, camera_poses, K):
  """
  Visualizes the reconstructed 3D points and camera frustums.

  Args:
      points3d (list): List of reconstructed 3D points.
      camera_poses (dict): Dictionary containing camera poses for each image.
      K (np.ndarray): Camera intrinsic matrix.
  """

  import cv2

  # Project 3D points onto the image plane of the first camera
  projected_points = np.dot(K, (points3d.T @ camera_poses[0][:3, :3]))[:2, :] / projected_points[2, :]

  # Draw points and camera frustums on a blank image
  image = np.zeros((480, 640, 3), dtype=np.uint8)
  for pt in projected_points.T:
      cv2.circle(image, (int(pt[0]), int(pt[1])), 3, (0, 255, 0), -1)

  # Assuming camera poses are represented by rotation and translation vectors
  for R, t in camera_poses.values():
      # Calculate camera frustum corners in world coordinates
      frustum_points = cv2.projectPoints(np.float32([[0.5, 0.5, 0.1], [-0.5, 0.5, 0.1],
                                                    [-0.5, -0.5, 0.1], [0.5, -0.5, 0.1],
                                                    [0.5, 0.5, 1.0], [-0.5, 0.5, 1.0],
                                                    [-0.5, -0.5, 1.0], [0.5, -0.5, 1.0]]),
                                         R, t, K)[0]

      # Project frustum corners onto the image plane
      frustum_points = np.dot(K, (frustum_points.T @ camera_poses[0][:3, :3]))[:2, :] / frustum_points[2, :]

      # Draw camera frustum
      cv2.polylines(image, [np.int32(frustum_points)], True, (255, 0, 0), 2)

  # Display the visualization
  cv2.imshow("3D Reconstruction Visualization", image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [20]:
def refine_matches_lowe(matches, descriptors1, descriptors2):
    """Refines feature matches using Lowe's ratio test.

    Args:
        matches (list): List of matches between keypoints.
        descriptors1 (list): List of descriptors for the first image.
        descriptors2 (list): List of descriptors for the second image.

    Returns:
        list: A list of refined matches that pass the Lowe's ratio test.
    """

    refined_matches = []
    ratio_thresh = 0.8  # Adjust threshold as needed

    for match in matches:
        # Get the two best matches for the query descriptor
        first_match = match
        second_match_idx = min(i for i in range(len(matches)) if i != match.queryIdx and matches[i].trainIdx == match.trainIdx)
        second_match = matches[second_match_idx]

        # Calculate the distance ratio
        distance_ratio = match.distance / second_match.distance

        # If the ratio is greater than the threshold, consider it a good match
        if distance_ratio > ratio_thresh:
            refined_matches.append(match)

    return refined_matches

def filter_matches_ransac(matches, keypoints1, keypoints2, camera_poses, ransac_thresh=3.0):
  """
  Filters feature matches using RANSAC (Random Sample Consensus).

  Args:
      matches (list): List of matches between keypoints.
      keyframes1 (list): List of keypoints for the first image.
      keyframes2 (list): List of keypoints for the second image.
      camera_poses (dict): Dictionary containing camera poses for each image.
      ransac_thresh (float, optional): Threshold for inlier distance in RANSAC. Defaults to 3.0.

  Returns:
      list: A list of inlier matches that pass the RANSAC test.
  """

  # Import necessary libraries (assuming OpenCV is already imported)
  import cv2

  inlier_matches = []
  if len(matches) > 4:  # Require at least 4 matches for RANSAC
      # Extract keypoint locations for all matches
      src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
      dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

      # Estimate homography (or fundamental matrix for uncalibrated cameras)
      model, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransac_thresh)

      # Filter inliers based on the homography and distance threshold
      for i, m in enumerate(matches):
          if mask[i][0]:  # Check if the match is an inlier
              # Project the source point back onto the second image using the homography
              projected_pt = cv2.perspectiveTransform(src_pts[i].reshape(-1, 1, 2), model)
              # Calculate the distance between the projected point and the matched point
              distance = np.linalg.norm(projected_pt[0][0] - dst_pts[i])
              # Add the inlier match if the distance is within a threshold
              if distance < ransac_thresh:
                  inlier_matches.append(m)

  return inlier_matches

def triangulate_points(inlier_matches, keypoints1, keypoints2, camera_poses):
  """
  Reconstructs 3D points from inlier matches using triangulation.

  Args:
      inlier_matches (list): List of inlier matches after RANSAC filtering.
      keyframes1 (list): List of keypoints for the first image.
      keyframes2 (list): List of keypoints for the second image.
      camera_poses (dict): Dictionary containing camera poses for each image.

  Returns:
      list: A list of reconstructed 3D points.
  """

  # Import libraries (assuming NumPy is already imported)
  import numpy as np

  # Initialize empty list for 3D points
  points3d = []

  for match in inlier_matches:
      # Extract corresponding keypoints from both images
      query_pt = keypoints1[match.queryIdx].pt
      train_pt = keypoints2[match.trainIdx].pt

      # Extract camera projection matrices from camera poses
      P1 = camera_poses[match.imgIdx1]  # Assuming camera poses are stored by image index
      P2 = camera_poses[match.imgIdx2]

      # Homogeneous coordinates for the corresponding keypoints
      q = np.array([query_pt[0], query_pt[1], 1.0])
      p = np.array([train_pt[0], train_pt[1], 1.0])

      # Triangulation using linear least squares solution
      A = np.vstack([np.cross(P1[2], P1[1] * p[0] - P1[0] * p[1]),
                     np.cross(P2[2], P2[1] * q[0] - P2[0] * q[1])])
      _, _, V = np.linalg.svd(A)
      X = V[-1, :] / V[-1, -1]  # Normalized homogeneous coordinates of the 3D point

      # Convert to non-homogeneous coordinates
      points3d.append(X[:3])

  return points3d

def poisson_surface_mesh(points3d):
  """
  Generates a surface mesh from a set of 3D points using Poisson surface reconstruction.

  Args:
      points3d (list): List of reconstructed 3D points.

  Returns:
      tuple: A tuple containing the vertices and faces of the surface mesh (or None if PyMesh is unavailable).
  """

  try:
    import pymesh
    verts = pymesh.utils.trimesh_to_vertex_list(np.array(points3d))
    mesh = pymesh.generate_poisson_mesh(verts)
    vertices = mesh.vertices
    faces = mesh.faces
    return vertices, faces
  except ImportError:
    print("PyMesh library not found. Surface meshing skipped.")
    return None

# Function for visualization using OpenCV (unchanged)
def visualize_reconstruction(points3d, camera_poses, K):
  """
  Visualizes the reconstructed 3D points and camera frustums.

  Args:
      points3d (list): List of reconstructed 3D points.
      camera_poses (dict): Dictionary containing camera poses for each image.
      K (np.ndarray): Camera intrinsic matrix.
  """

  import cv2

  # Project 3D points onto the image plane of the first camera
  projected_points = np.dot(K, (points3d.T @ camera_poses[0][:3, :3]))[:2, :] / projected_points[2, :]

  # Draw points and camera frustums on a blank image
  image = np.zeros((480, 640, 3), dtype=np.uint8)
  for pt in projected_points.T:
      cv2.circle(image, (int(pt[0]), int(pt[1])), 3, (0, 255, 0), -1)

  # Assuming camera poses are represented by rotation and translation vectors
  for R, t in camera_poses.values():
      # Calculate camera frustum corners in world coordinates
      frustum_points = cv2.projectPoints(np.float32([[0.5, 0.5, 0.1], [-0.5, 0.5, 0.1],
                                                    [-0.5, -0.5, 0.1], [0.5, -0.5, 0.1],
                                                    [0.5, 0.5, 1.0], [-0.5, 0.5, 1.0],
                                                    [-0.5, -0.5, 1.0], [0.5, -0.5, 1.0]]),
                                         R, t, K)[0]

      # Project frustum corners onto the image plane
      frustum_points = np.dot(K, (frustum_points.T @ camera_poses[0][:3, :3]))[:2, :] / frustum_points[2, :]

      # Draw camera frustum
      cv2.polylines(image, [np.int32(frustum_points)], True, (255, 0, 0), 2)

  # Display the visualization
  cv2.imshow("3D Reconstruction Visualization", image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

# Assuming you have your feature matching, camera poses, and intrinsic matrix K
def detect_and_compute_features(image):
  """
  Detects keypoints and computes descriptors for a given image.

  Args:
      image (np.ndarray): The input image as a NumPy array.

  Returns:
      tuple: A tuple containing the list of keypoints and the list of descriptors.
  """

  # Import libraries
  import cv2

  # Convert image to grayscale (if necessary for your chosen detector)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Create a SIFT or SURF detector (replace with your preference)
  sift = cv2.xfeatures2d.SIFT_create()

  # Detect keypoints and compute descriptors
  keypoints, descriptors = sift.detectAndCompute(gray, None)

  return keypoints, descriptors
# Extract keypoints and descriptors from the first and second images (replace with your implementation)
keypoints1, descriptors1 = detect_and_compute_features(image1)
keypoints2, descriptors2 = detect_and_compute_features(image2)

# Refine feature matches using Lowe's ratio test
refined_matches = refine_matches_lowe(matches, descriptors1, descriptors2)

# Filter inlier matches using RANSAC
inlier_matches = filter_matches_ransac(refined_matches, keypoints1, keypoints2, camera_poses)

# Triangulate 3D points from inlier matches
points3d = triangulate_points(inlier_matches, keypoints1, keypoints2, camera_poses)

# Generate surface mesh (if PyMesh is available)
vertices, faces = poisson_surface_mesh(points3d)

# Visualize the reconstructed scene
visualize_reconstruction(points3d, camera_poses, K)

NameError: name 'image1' is not defined